In [2]:
# Import necessary Libraries
import pandas as pd
from lightgbm import LGBMClassifier


In [5]:

train = pd.read_csv("/Users/nathanzlomke/Documents/GitHub/ban5753-fall2023-rxdata/Humana-Mays Competition/Humana_Confidential_Analytics/model_data10-2.csv")  #took off encoding="ISO-8859-1" as it was causing glitching
train = train.drop(columns={'Date Duration'}, axis=1)
train.shape
train.columns

test = pd.read_csv("/Users/nathanzlomke/Documents/GitHub/ban5753-fall2023-rxdata/Humana-Mays Competition/Humana_Confidential_Analytics/CORRECTED_CLEAN_HOLDOUT_10-3.csv")
#None of the below 3 features in holdout file, so need to add as columns = 0 for model to work
test['race_North American Native']= 0
test['util_cat__IP_LTACH'] = 0
test['util_cat__IP_MHSA'] = 0
test.shape
test_new = test
test_new['tgt_ade_dc_ind']= 0


In [6]:


# Define X and y for training and testing
x_train = train.drop(columns=['tgt_ade_dc_ind'], axis=1)
y_train = train['tgt_ade_dc_ind']
x_test = test_new.drop(columns=['tgt_ade_dc_ind'], axis=1)
y_test = test_new['tgt_ade_dc_ind']


In [7]:
#This section uses an algorithm RandomSearchCV to optimize the model parameters for the model to use in the next section


#https://www.youtube.com/shorts/AOLfcy9FUjs
from sklearn.model_selection import GridSearchCV
seed=0
lgbm_params = {
    "n_estimators":[700],
    "learning_rate":[0.1],
    "num_leaves": [12]
}

lgmc = LGBMClassifier(seed=seed)
#gridsearch is too slow, online recommended using randmomized search
'''grid_search = GridSearchCV(
    estimator=lgmc,
    param_grid = lgbm_params,
    n_jobs=-1, cv=3,
    scoring='roc_auc',
    error_score = 0)
grid_result = grid_search.fit(x_train, y_train)

final_model = lgmc.set_params(**grid_result.best_params_)
final_model.fit(x_train, y_train)
y_pred = final_model.predict(x_test)

lgmc.score(x_test,y_test)
'''

from sklearn.model_selection import RandomizedSearchCV
rand = RandomizedSearchCV(LGBMClassifier(seed=seed), lgbm_params, n_iter=10, cv=3, scoring='roc_auc', random_state=1)
rand.fit(x_train,y_train)
rand.best_score_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 511820, number of negative: 2073937
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.218246 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2054
[LightGBM] [Info] Number of data points in the train set: 2585757, number of used features: 80
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.197938 -> initscore=-1.399231
[LightGBM] [Info] Start training from score -1.399231
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 511820, number of negative: 2073937
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.209315 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set 

0.44150865269752226

In [8]:
rand.best_params_
#{'num_leaves': 12, 'n_estimators': 500, 'learning_rate': 0.1}
#***BEST is {'num_leaves': 12, 'n_estimators': 700, 'learning_rate': 0.1}

{'num_leaves': 12, 'n_estimators': 700, 'learning_rate': 0.1}

In [9]:

# Create and train the LightGBM model
rand = LGBMClassifier(seed=seed)
rand.fit(x_train, y_train)

# Predict the target variable
pred = rand.predict(x_test)
print(pred)

# Evaluate model performance (consider using additional metrics)
accuracy = rand.score(x_test, y_test)
print("Accuracy:", accuracy)
print(y_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 767730, number of negative: 3110906
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.327447 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2103
[LightGBM] [Info] Number of data points in the train set: 3878636, number of used features: 80
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.197938 -> initscore=-1.399231
[LightGBM] [Info] Start training from score -1.399231
[1 1 1 ... 1 1 1]
Accuracy: 0.9009427369134205
0         0
1         0
2         0
3         0
4         0
         ..
608118    0
608119    0
608120    0
608121    0
608122    0
Name: tgt_ade_dc_ind, Length: 608123, dtype: int64


In [10]:
#Printing the scored results and joining to test dataframe


#test_scored.head()
pred = pd.DataFrame(pred)
pred = pred.rename(columns={0:'tgt_ade_dc_ind'})
test_scored = x_test.join(pred)
#test_scored = test_new.join(y_test, how = 'left', lsuffix='left', rsuffix='right')


In [11]:
pred.shape

(608123, 1)

In [12]:
short_test_scored = test_scored[['cleaned_id','tgt_ade_dc_ind']]
short_test_scored.head()

grouped = short_test_scored.groupby('cleaned_id')
average_id = grouped.mean()
average_id.shape
average_id.head()


,tgt_ade_dc_ind
cleaned_id,
1000261259,0.096987
1001691228,0.000000
1002060555,0.487302
1003935675,0.000000
1004837018,0.111718


In [15]:
short_test_scored['cleaned_id'].nunique()
x_test['cleaned_id'].nunique()

420

In [16]:
#Export to CSV
average_id.to_csv("scoring_10_3.csv")
